# AI CNN Model for Detecting Melanomas and Melanocytic Growths

*This model script was run in Google Colab to access GPUs for training.*

# Data Collection

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/CODE/EDU/Udacity-DL-Nano-DermoCNN

/content/drive/MyDrive/CODE/EDU/Udacity-DL-Nano-DermoCNN


In [12]:
!pwd

/content/drive/My Drive/CODE/EDU/Udacity-DL-Nano-DermoCNN


In [13]:
!mkdir data
%cd data

/content/drive/MyDrive/CODE/EDU/Udacity-DL-Nano-DermoCNN/data


In [14]:
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip

--2021-07-26 23:52:40--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.116.224
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.116.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5736557430 (5.3G) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]   5.34G  37.0MB/s    in 2m 30s  

2021-07-26 23:55:10 (36.4 MB/s) - ‘train.zip’ saved [5736557430/5736557430]



In [15]:
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip

--2021-07-26 23:56:08--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.116.216
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.116.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 864538487 (824M) [application/zip]
Saving to: ‘valid.zip’

valid.zip           100%[===================>] 824.49M  17.9MB/s    in 38s     

2021-07-26 23:56:46 (21.6 MB/s) - ‘valid.zip’ saved [864538487/864538487]



In [16]:
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/test.zip

--2021-07-26 23:58:20--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/test.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.120.8
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.120.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528640507 (5.1G) [application/zip]
Saving to: ‘test.zip’

test.zip            100%[===================>]   5.15G  33.3MB/s    in 2m 51s  

2021-07-27 00:01:12 (30.8 MB/s) - ‘test.zip’ saved [5528640507/5528640507]



In [ ]:
!unzip test.zip

In [18]:
!unzip train.zip &> /dev/null

In [19]:
!unzip valid.zip &> /dev/null

In [20]:
!rm test.zip ; rm train.zip ; rm valid.zip

In [25]:
%cd ..

/content/drive/My Drive/CODE/EDU/Udacity-DL-Nano-DermoCNN


## Data Inspection

In [26]:
#Walk through all directories and files and gather minimum dimension of each image
import os
import PIL
from PIL import Image

image_size_dict = {}

for (root, dirs, files) in os.walk('data', topdown=True):
    if len(files) > 0 and '.DS_Store' not in files:
        print('Now iterating through files in', root)
        file_count = 0
        for i in files:
#             print(i)
            image = PIL.Image.open(f'{root}/{i}')
            width, height = image.size
            min_dimension = min([width, height])
#             print(min_dimension)
            image_size_dict[i] = min_dimension
            file_count +=1
        print(f'Found {file_count} files in {root} \n')

Now iterating through files in data/test/melanoma
Found 117 files in data/test/melanoma 

Now iterating through files in data/test/nevus
Found 393 files in data/test/nevus 

Now iterating through files in data/test/seborrheic_keratosis
Found 90 files in data/test/seborrheic_keratosis 

Now iterating through files in data/train/melanoma
Found 374 files in data/train/melanoma 

Now iterating through files in data/train/nevus
Found 1372 files in data/train/nevus 

Now iterating through files in data/train/seborrheic_keratosis
Found 254 files in data/train/seborrheic_keratosis 

Now iterating through files in data/valid/melanoma
Found 30 files in data/valid/melanoma 

Now iterating through files in data/valid/nevus
Found 78 files in data/valid/nevus 

Now iterating through files in data/valid/seborrheic_keratosis
Found 42 files in data/valid/seborrheic_keratosis 



In [27]:
print('Smallest image dimension is', min(image_size_dict.values()), 'pixels')

Smallest image dimension is 540 pixels


In [4]:
classes = {'melanoma': 'malignant', 'nevus': 'benign', 'seborrheic_keratosis':'benign'}
print('AI will predict image as one of the following: \n')
print([(x, y) for (x, y) in classes.items()],'\n')

AI will predict image as one of the following: 

[('melanoma', 'malignant'), ('nevus', 'benign'), ('seborrheic_keratosis', 'benign')] 



## Model Prep

### Load Data

In [5]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

## Specify appropriate transforms, and batch_sizes

### CUDA check
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available. Training on CPU...')
else:
    print('CUDA is available. Training on GPU...')
    
### Data Loading
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 64

train_transform = transforms.Compose([
    transforms.Resize(size=320),
    transforms.CenterCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

valid_and_test_transform = transforms.Compose([
    transforms.Resize(size=320),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])


train_dataset = datasets.ImageFolder("data/train", transform=train_transform)
valid_dataset = datasets.ImageFolder("data/valid", transform=valid_and_test_transform)
test_dataset = datasets.ImageFolder("data/test", transform=valid_and_test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
    num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, 
    num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
    num_workers=num_workers)

CUDA is available. Training on GPU...


### Build Model

In [6]:
model_transfer = models.inception_v3(pretrained=True)

# print out the model structure
print(model_transfer)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth



Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), str

In [7]:
print(model_transfer.fc.in_features) 
print(model_transfer.fc.out_features)

2048
1000


In [8]:
# Freeze training for certain layers except AuxLogits, Mixed_7's, and Final FC
params_to_update = []

for name, module in model_transfer.named_children():
    if name == 'AuxLogits':
#         for param in module.conv0.parameters():
#             param.requires_grad = False
#         for param in module.conv1.parameters():
#             param.requires_grad = False
        for param in module.parameters():
            params_to_update.append(param)
    elif name == 'fc':
        for param in module.parameters():
            params_to_update.append(param)
    elif name == 'Mixed_7a':
        for param in module.parameters():
            params_to_update.append(param)
    elif name == 'Mixed_7b':
        for param in module.parameters():
            params_to_update.append(param)
    elif name == 'Mixed_7c':
        for param in module.parameters():
            params_to_update.append(param)
    else:
        for param in module.parameters():
            param.requires_grad = False

In [9]:
#Set Classes
import torch.nn as nn
num_classes = len(classes.keys())
model_transfer.AuxLogits.fc = nn.Linear(768, num_classes)
model_transfer.fc = nn.Linear(2048, num_classes)

In [10]:
if train_on_gpu:
    model_transfer = model_transfer.cuda()
    print('Training on GPU')
else:
    print('Training on CPU')

Training on GPU


In [11]:
#Specify criterion and optimizers
import torch.optim as optim
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(params_to_update, lr=0.05)

### Train Model

In [17]:
import numpy as np
from tqdm.notebook import tqdm

def inception_train(n_epochs, loaders, model, optimizer, criterion, train_on_gpu, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    for epoch in tqdm(range(1, n_epochs+1), desc='Training: '):
        print('Now in epoch ', epoch, '...')
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            outputs, aux_outputs = model(data)
            loss1 = criterion(outputs, target)
            loss2 = criterion(aux_outputs, target)
            loss = loss1 + 0.4*loss2
            loss.backward()
            optimizer.step()
#             train_loss += loss.item()*data.size(0)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
    
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            
            optimizer.zero_grad()
            
            with torch.no_grad():
                output = model(data)
            loss = criterion(output, target)
#             valid_loss += loss.item()*data.size(0)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

#         train_loss = train_loss/len(loaders['train'])
#         valid_loss = valid_loss/len(loaders['valid'])
    
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), 'model_transfer.pt')
            valid_loss_min = valid_loss
        
    # return trained model
    return model

In [18]:
loaders_transfer = {
    'train': train_loader,
    'valid': valid_loader,
    'test': test_loader
}

# train the model
model_transfer = inception_train(15, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, train_on_gpu, 'model_transfer.pt')

Now in epoch  1 ...
Epoch: 1 	Training Loss: 1.168917 	Validation Loss: 0.820124
Validation loss decreased (inf --> 0.820124).  Saving model ...
Now in epoch  2 ...
Epoch: 2 	Training Loss: 1.002983 	Validation Loss: 0.805819
Validation loss decreased (0.820124 --> 0.805819).  Saving model ...
Now in epoch  3 ...
Epoch: 3 	Training Loss: 0.938094 	Validation Loss: 0.802410
Validation loss decreased (0.805819 --> 0.802410).  Saving model ...
Now in epoch  4 ...
Epoch: 4 	Training Loss: 0.856422 	Validation Loss: 0.685580
Validation loss decreased (0.802410 --> 0.685580).  Saving model ...
Now in epoch  5 ...
Epoch: 5 	Training Loss: 0.790956 	Validation Loss: 0.767669
Now in epoch  6 ...
Epoch: 6 	Training Loss: 0.756284 	Validation Loss: 0.713760
Now in epoch  7 ...
Epoch: 7 	Training Loss: 0.711925 	Validation Loss: 0.770801
Now in epoch  8 ...
Epoch: 8 	Training Loss: 0.670911 	Validation Loss: 0.728975
Now in epoch  9 ...
Epoch: 9 	Training Loss: 0.610373 	Validation Loss: 0.782201


In [19]:
# load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

<All keys matched successfully>

### Test Model

In [20]:
def test(loaders, model, criterion, train_on_gpu):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [21]:
test(loaders_transfer, model_transfer, criterion_transfer, train_on_gpu)

Test Loss: 0.780688


Test Accuracy: 72% (435/600)


## Find class probabilities

In [26]:
import csv
!pwd

/content/drive/My Drive/CODE/EDU/Udacity-DL-Nano-DermoCNN


### Define custom loader

In [32]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

# EXAMPLE USAGE:
# instantiate the dataset and dataloader
data_dir = 'data/test'
dataset = ImageFolderWithPaths(data_dir, transform=valid_and_test_transform) # our custom dataset
# dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, 
#     num_workers=num_workers)
dataloader = torch.utils.data.DataLoader(dataset)

Find relevant class indices

In [51]:
print(dataset.classes)
task_1_index = dataset.classes.index('melanoma')
task_2_index = dataset.classes.index('seborrheic_keratosis')
print(task_1_index, task_2_index)

['melanoma', 'nevus', 'seborrheic_keratosis']
0 2


Evaluate code to return probabilities before implementation

In [54]:
model_transfer.eval()
for batch_idx, (data, target, path) in enumerate(dataloader):
    # move to GPU
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_transfer(data)
    sm = torch.nn.Softmax(dim=1)
    probabilities = sm(output)
    probabilities = probabilities.cpu().detach().numpy()
    print(probabilities)
    print(probabilities[0])
    print(probabilities[0][task_1_index])
    print(probabilities[0][task_2_index])
    print(probabilities.sum(1))
    break

[[0.14256275 0.67881584 0.17862146]]
[0.14256275 0.67881584 0.17862146]
0.14256275
0.17862146
[1.]


In [55]:
def probs(loaders, model, criterion, train_on_gpu, header, task_1_index, task_2_index):
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()

    data_points = []

    for batch_idx, (data, target, path) in enumerate(loaders):
        # move to GPU
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the probabilities
        probabilities = criterion(output) 
        probabilities = probabilities.cpu().detach().numpy()
        task_1_prob = probabilities[0][task_1_index]
        task_2_prob = probabilities[0][task_2_index]
        data_points.append([path[0], task_1_prob, task_2_prob])

    with open('results.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(header)

        # write multiple rows
        writer.writerows(data_points)


In [56]:
prob_criterion = torch.nn.Softmax(dim=1)
header = ['Id', 'task_1', 'task_2']
probs(dataloader, model_transfer, prob_criterion, train_on_gpu, header, task_1_index, task_2_index)